In [17]:
from iohub.ngff import open_ome_zarr
from natsort import natsorted
from glob import glob
from pathlib import Path 
import torch
from torch.utils.data import Dataset
from scipy.ndimage import measurements
from scipy.ndimage import center_of_mass
import numpy as np

zarr_dir = "/mnt/efs/dlmbl/G-et/data/neuromast/Dataset/"
# defines input zarr file name with the zarr file structure
zarr_file = 'structured_celltype_classifier_data.zarr/*/*/*'
position_paths = natsorted(glob(zarr_dir + zarr_file))
# print(position_paths)


In [3]:
print("Number of positions: ", len(position_paths))

Number of positions:  1000


In [4]:
dataset = open_ome_zarr(position_paths[0], mode="r")
dataset.data.shape
all_chan = dataset.channel_names
chan = 'celltypes'
all_chan.index(chan)

3

In [5]:
class NeuromastDataset(Dataset):
    def __init__(self, file_path, class_channel: str = 'celltypes'):
        self.file_path = file_path
        #get the class index
        dataset = open_ome_zarr(self.file_path[0], mode="r")
        all_chan = dataset.channel_names
        self.c = all_chan.index(class_channel)
        

    def __len__(self):
        return len(self.file_path)

    def __getitem__(self, idx):
        dataset = open_ome_zarr(position_paths[idx], mode="r")
        image = dataset.data[:,0:self.c,:,:,:]
        label = dataset.data[:,self.c:self.c+1,:,:,:]
        return image, label

        


In [6]:
neuromast = NeuromastDataset(position_paths, class_channel='celltypes')
print(neuromast[0][0].shape)
print(neuromast[0][1].shape)

(1, 3, 73, 1024, 1024)
(1, 1, 73, 1024, 1024)


In [21]:

dataset = open_ome_zarr(position_paths[0], mode="r")


all_chan = dataset.channel_names
chan = 'celltypes'
c = all_chan.index(chan)

image = dataset.data[:,0:c,:,:,:]
celltype = dataset.data[0,c:c+1,:,:,:]
segmented_data = dataset.data[0,2]




/tmp/ipykernel_135839/513073290.py:2: DeprecationWarning: Please import `binary_dilation` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.morphology import binary_dilation


In [22]:
segment_labels = np.unique(segmented_data)
segment_labels = segment_labels[segment_labels != 0]  # Exclude background

centroids = {}

# Calculate the centroid for each segment
for label in segment_labels:
    # Get a binary mask of the current segment
    segment_mask = segmented_data == label
    
    # Compute the centroid
    coords = np.array(np.nonzero(segment_mask))
    print("coords",coords)  # Get the coordinates of the segment
    centroid = np.mean(coords, axis=1)
    print("centroid", centroid)   # Calculate the mean along each axis (Z, Y, X)
    centroids[label] = centroid
    
    # Store the centroid
    centroids[label] = centroid
    if  label ==2:
        break

# Now, centroids dictionary holds the centroid for each segment label
print(centroids)

coords [[  5   5   5 ...  44  44  44]
 [359 359 359 ... 385 386 386]
 [508 509 510 ... 512 505 506]]
centroid [ 22.28359574 364.65511098 507.8500329 ]
coords [[  5   5   5 ...  46  46  46]
 [397 397 397 ... 383 384 384]
 [453 454 455 ... 442 438 439]]
centroid [ 21.6632611  392.1020675  445.27676021]
{np.float32(1.0): array([ 22.28359574, 364.65511098, 507.8500329 ]), np.float32(2.0): array([ 21.6632611 , 392.1020675 , 445.27676021])}
